<a href="https://colab.research.google.com/github/DataTak/finacial_data_analysis/blob/main/%EB%84%A4%EC%9D%B4%EB%B2%84_%EA%B2%BD%EC%A0%9C%EB%89%B4%EC%8A%A4%ED%81%AC%EB%A1%A4%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 네이버 경제뉴스 크롤링
- 네이버 증권의 뉴스 탭의 최신 10일치 기사를 수집하여 데이터 프레임으로 생성, 데이터프레임을 article.csv 로 저장하기

- 네이버 증권 뉴스 탭<br>
https://finance.naver.com/news/mainnews.naver?date=2024-10-25
- 날짜별로 페이지가 구분되어 있고, 날짜를 변경할 경우, URL의 date의 속성값 변경
- 각 날짜에는 페이지네이션으로 페이지가 변경됨 <br>
https://financenaver.com/news/mainnews.naver?date=2024-10-25&page=3
- 페이지네이션은 고려하지 않고, 첫 페이지에 있는 데이터만 수집


- 데이터 프레임 구조
    - 기사제목 / 본문 URL / 기사본문 / 날짜  로 구성
- CSV 파일로 저장시, 인코딩(euc-kr 또는 cp949) 하여 저장할 것

In [69]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
from datetime import datetime, timedelta


In [74]:
def extract_article_data(articles):
    # 데이터를 저장할 리스트
    data = []

    # 각 기사의 링크에서 article_id와 office_id를 추출
    for article in articles:
        a_tag = article.find('a')

        if a_tag and 'href' in a_tag.attrs:
            href = a_tag['href']

            # 필요한 부분 추출 (article_id, office_id)
            query_params = href.split('?')[1]  # ? 뒤의 쿼리 파라미터
            params = dict(param.split('=') for param in query_params.split('&'))

            article_id = params.get('article_id')
            office_id = params.get('office_id')


            # 최종 URL 생성
            final_url = f"https://n.news.naver.com/mnews/article/{office_id}/{article_id}"


            # 해당 기사 본문 뉴스 가지고 오기
            response = requests.get(final_url)
            soup = BeautifulSoup(response.text, 'lxml')
            article_text = soup.find('div', {'class': 'newsct_article _article_body'}).text.strip()


            # 해당 기사 날짜 가지고 오기
            news_date = soup.find('span', {'class': 'media_end_head_info_datestamp_time _ARTICLE_DATE_TIME'})['data-date-time']

            # 데이터 저장
            data.append({
                '기사 제목': a_tag.get_text().strip(),
                '본문 URL': final_url,
                '본문': article_text,
                '날짜': news_date
            })

            time.sleep(0.5)

    # 데이터프레임 생성
    article_df = pd.DataFrame(data)

    return article_df


In [71]:
# 날짜 입력 받기
input_date = '2024-10-25'
start_date = datetime.strptime(input_date, '%Y-%m-%d')

# 빈 데이터프레임 생성
all_articles_df = pd.DataFrame()

# 과거 10일간의 뉴스 페이지 가져오기
for i in range(10):
    # 현재 날짜 계산 (입력 날짜부터 과거로 하루씩 감소)
    current_date = start_date - timedelta(days=i)
    formatted_date = current_date.strftime('%Y-%m-%d')

    # URL에 현재 날짜를 포함시켜 요청
    url = f'https://finance.naver.com/news/mainnews.naver?date={formatted_date}'
    response = requests.get(url)

    # 상태 코드 확인
    if response.status_code == 200:
        print(f"{formatted_date} 페이지 접근 성공")

        soup = BeautifulSoup(response.text, 'lxml')
        articles = soup.find_all('dd', {'class': 'articleSubject'})

        # 데이터 추출
        article_df = extract_article_data(articles)

        # 날짜별 df 쌓기
        all_articles_df = pd.concat([all_articles_df, article_df], ignore_index=True)

    else:
        print(f"{formatted_date} 페이지 접근 실패 - 상태 코드: {response.status_code}")

2024-10-25 페이지 접근 성공
2024-10-24 페이지 접근 성공
2024-10-23 페이지 접근 성공
2024-10-22 페이지 접근 성공
2024-10-21 페이지 접근 성공
2024-10-20 페이지 접근 성공
2024-10-19 페이지 접근 성공
2024-10-18 페이지 접근 성공
2024-10-17 페이지 접근 성공
2024-10-16 페이지 접근 성공


In [72]:
all_articles_df

,기사 제목,본문 URL,본문,날짜
0,“찬바람 불길래 믿고 사놨는데”…오를 놈은 따로 있다는 보험주,https://n.news.naver.com/mnews/article/009/000...,\n\n계약 해지떄 돌려줄 준비금회계기준 바뀌며 더 필요해소형 보험주 배당여력 타격...,2024-10-25 20:23:07
1,AI 열풍에도 주가 부진한 국내 IT사…'수혜주가 아니라 피해주',https://n.news.naver.com/mnews/article/015/000...,\n\n\n\n\n\n네이버·카카오를 비롯한 국내 인공지능(AI) 관련주 주가가 올...,2024-10-25 20:05:10
2,고 이건희 4주기 '삼성 위기론' 속 조용한 추모…이재용 쇄신 카드는?,https://n.news.naver.com/mnews/article/057/000...,\n\n\t\t\t【 앵커멘트 】 고 이건희 삼성 선대회장의 4주기를 맞아 이재용...,2024-10-25 20:01:07
3,흑백요리사로 대박 난 백종원…무려 4900억 ‘잭팟’ 터졌다는데,https://n.news.naver.com/mnews/article/009/000...,\n\n\n\n\n\n ‘흑백요리사’ 백종원 심사위원. [사진 출처 = 넷플릭스]백...,2024-10-25 18:59:06
4,'20만닉스' 컴백한 날…청산가치 밑으로 추락한 삼전,https://n.news.naver.com/mnews/article/015/000...,"\n\n블랙먼데이 이후 엇갈린 운명외국인, 33거래일 연속 순매도주가, 주당순자산가...",2024-10-25 18:10:20
...,...,...,...,...
195,수익 악화·감원 … 美보험·약국株 아프다,https://n.news.naver.com/mnews/article/009/000...,\n\n美 최대 보험 유나이티드헬스암울한 실적에 주가 8% 하락대형 드러그스토어 체...,2024-10-16 17:26:09
196,"‘5만전자’ 털썩…ASML發 반도체 된서리 맞은 코스피, 2610대로 후퇴 [투자360]",https://n.news.naver.com/mnews/article/016/000...,\n\n외인·기관 ‘팔자’…삼성전자 등 반도체주 일제히 하락트럼프 당선 가능성 확대...,2024-10-16 17:24:11
197,중동 불안에 출렁이는 유가…원유 ETN도 '롤러코스터' [이슈+],https://n.news.naver.com/mnews/article/015/000...,"\n\n중동 정세에 국제유가 변동성 확대원유 ETN 수익률도 등락 이어져""유가 하락...",2024-10-16 17:20:19
198,"세아제강, 회사채 수요예측서 1조 넘게 모아…A급 연이어 흥행[마켓인]",https://n.news.naver.com/mnews/article/018/000...,"\n\n세아제강, 800억 모집에 1조250억 주문전 트랜치 언더금리서 모집LS·H...",2024-10-16 17:14:14


# 교수님 코드

# 개요
- 네이버 증권 `뉴스` 탭의 3일치 기사를 수집하여 데이터 프레임으로 생성하고 파일로 저장
- 사이트 주소: https://finance.naver.com/news/mainnews.naver?date=2024-10-25

# 신문기사 목록 불러오기

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

url = 'https://finance.naver.com/news/mainnews.naver?date=2024-10-25'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'lxml')


- 데이터 프레임 구조  
    - **기사제목**:  `<dd class="articleSubject">`안에   텍스트  
    - **본문 URL**: `<dd class="articleSubject">`   안에 `<a>` 태그  
    - **기사 본문**: `<div id="newsct_article"   class="newsct_article _article_body">`  
    - **날짜**: URL에서 가져오기

In [ ]:
# 기사제목 리스트 가져오기
titles = soup.find_all('dd', {'class':'articleSubject'})
titles[0]

In [ ]:
# 첫번째 기사의 기사 제목
titles[0].text.strip()

# 상세 뉴스 본문 읽어오기

## 상세 뉴스 URL 읽어오기

In [ ]:
# 첫번째 기사의 상세페이지 URL
titles[0].find('a').get('href')

In [ ]:
news_url = 'https://finance.naver.com' + titles[0].find('a').get('href')
news_url

In [ ]:
# 상세 뉴스페이지
news_response = requests.get(news_url)

news_soup = BeautifulSoup(news_response.text, 'lxml')
news_soup

In [ ]:
news_url2 = news_soup.find('script').text.split("'")[1]

## 상세 뉴스 페이지 읽어오기

In [ ]:
# 상세 뉴스페이지 재도전
news_response2 = requests.get(news_url2)

news_soup = BeautifulSoup(news_response2.text, 'lxml')
news_soup

## 상세 뉴스 페이지 본문 파싱하기

In [ ]:
# 뉴스 본문
# 기사 본문: <div id="newsct_article" class="newsct_article _article_body">
news_soup.find('div', {'class':'newsct_article _article_body'}).text.strip()

# 반복문 사용하여 함수로 생성하기

In [ ]:
import datetime
import time

In [ ]:
def get_news_items(html):
  # 기사제목 리스트 가져오기
  titles = html.find_all('dd', {'class':'articleSubject'})
  title_list = []
  url_list = []
  article_list = []

  for t in titles:
    # 기사의 제목
    title = t.text.strip()

    # 기사의 상세페이지 URL
    news_url = 'https://finance.naver.com' + t.find('a').get('href')

    # 상세페이지 request
    news_response = requests.get(news_url)
    news_soup = BeautifulSoup(news_response.text, 'lxml')

    # 리다이렉트 주소를 파싱
    news_url2 = news_soup.find('script').text.split("'")[1]

    # 상세 뉴스페이지 request 재도전
    news_response2 = requests.get(news_url2)
    news_soup = BeautifulSoup(news_response2.text, 'lxml')

    # 신문기사 본문 파싱
    article = news_soup.find('div', {'class':'newsct_article _article_body'}).text.strip()

    # 리스트에 값 채우기
    title_list.append(title)
    url_list.append(news_url2)
    article_list.append(article)

  df = pd.DataFrame({'기사제목': title_list, '본문url': url_list, '기사본문': article_list})
  return df

In [ ]:
news_df = pd.DataFrame()

for i in range(3):
    # 오늘 날짜 기준 i일 이전 날짜 구하기
    date= datetime.date.today() - datetime.timedelta(days = i)
    url = f'https://finance.naver.com/news/mainnews.naver?date={date}'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    time.sleep(1)

    # 해당일 신문기사 스크랩 데이터프레임 반환
    temp_df = get_news_items(soup)
    temp_df['date'] = date
    news_df = pd.concat([news_df, temp_df], ignore_index=True).copy()

In [ ]:
news_df.to_csv('article.csv')